### Perspective PS7

#### 1. Classifier "horse" race

In [1]:
# a). 
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, mean_squared_error
auto = pd.read_csv("Auto.csv", na_values='?')
auto = auto.dropna()

In [2]:
auto.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight            int64
acceleration    float64
year              int64
origin            int64
name             object
dtype: object

In [3]:
mpg_high = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0).astype('category')
auto['horsepower'] = auto['horsepower'].astype('float64')
auto['mpg_high'] = mpg_high
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


In [4]:
X = auto[['cylinders','displacement','horsepower','weight','acceleration','year','origin']]
Xvals = X.values
yvals = auto.mpg_high.values
X.head()

,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,8,307.0,130.0,3504,12.0,70,1
1,8,350.0,165.0,3693,11.5,70,1
2,8,318.0,150.0,3436,11.0,70,1
3,8,304.0,150.0,3433,12.0,70,1
4,8,302.0,140.0,3449,10.5,70,1


In [5]:
from sklearn.model_selection import train_test_split, KFold
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvals)

MSE_vec_kf = np.zeros(k)
e_0 = np.zeros(k)
e_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    err0 = ((y_pred==0)*(np.abs(y_test-y_pred))).sum() / (y_pred==0).sum()
    e_0[k_ind] = (err0)
    err1 = ((y_pred==1)*(np.abs(y_test-y_pred))).sum() / (y_pred==1).sum()
    e_1[k_ind] = (err1)
    print('Error rate (low, 0): ', err0)
    print('Error rate (high, 1): ', err1)
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf)
print('The average error rate of mpg_high = 0 is', e_0.mean())
print('The average error rate of mpg_high = 1 is', e_1.mean())

k index= 0
MSE for test set 0  is 0.0918367346939
Error rate (low, 0):  0.0576923076923
Error rate (high, 1):  0.130434782609

              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

k index= 1
MSE for test set 1  is 0.102040816327
Error rate (low, 0):  0.122448979592
Error rate (high, 1):  0.0816326530612

              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

k index= 2
MSE for test set 2  is 0.132653061224
Error rate (low, 0):  0.152173913043
Error rate (high, 1):  0.115384615385

              precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0

In [6]:
# b). 
from sklearn.ensemble import RandomForestClassifier
Xvals = auto[['cylinders','displacement','horsepower','weight','acceleration','year','origin']].values
rf = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                           oob_score=True, random_state=25)
rf.fit(Xvals, yvals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [7]:
rf_pred = rf.oob_decision_function_
MSE_rf = pd.DataFrame({'ypred' : rf_pred.T[1], 'yvals': yvals})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)
MSE_model = mean_squared_error(MSE_rf['yvals'], MSE_rf['ypred'])
print('The MSE of the random forest model is:',MSE_model)
MSE_rf_0 = MSE_rf[MSE_rf['ypred'] < 0.5]
MSE_0 = mean_squared_error(MSE_rf_0['yvals'], MSE_rf_0['ypred'])
print('The MSE of the mpg_high = 0 is:',MSE_0)
MSE_rf_1 = MSE_rf[MSE_rf['ypred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_rf_1['yvals'], MSE_rf_1['ypred'])
print('The MSE of the mpg_high = 1 is:',MSE_1)

The MSE of the random forest model is: 0.0714285714286
The MSE of the mpg_high = 0 is: 0.0578947368421
The MSE of the mpg_high = 1 is: 0.0841584158416


In [14]:
# c). 
from sklearn.svm import SVC
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvals)
error_0 = np.zeros(k)
error_1 = np.zeros(k)
MSE_vec_kf = np.zeros(k)
k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred  
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    e_0 = ((y_pred==0)* error).sum() / (y_pred==0).sum()
    e_1 = ((y_pred==1)* error).sum() / (y_pred==1).sum()
    error_0[k_ind] = (e_0)
    error_1[k_ind] = (e_1)
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('Error rate (low, 0): ', e_0)
    print('Error rate (high, 1): ', e_1)
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1


k index= 0
MSE for test set 0  is 0.540816326531
Error rate (low, 0):  0.0
Error rate (high, 1):  0.552083333333

              precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

k index= 1
MSE for test set 1  is 0.520408163265
Error rate (low, 0):  0.520408163265
Error rate (high, 1):  nan

              precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

k index= 2
MSE for test set 2  is 0.520408163265
Error rate (low, 0):  0.53125
Error rate (high, 1):  0.0

              precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

k index= 3
MSE fo

/Users/ivychen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/Users/ivychen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print()
print('test estimate MSE k-fold=', MSE_kf)

error_0 = error_0[~np.isnan(error_0)]
error_1 = error_1[~np.isnan(error_1)]
print('The average error rate of mpg_high = 0 is', error_0.mean())
print('The average error rate of mpg_high = 1 is', error_1.mean())


test estimate MSE k-fold= 0.507653061224
The average error rate of mpg_high = 0 is 0.381194110709
The average error rate of mpg_high = 1 is 0.184027777778


In [17]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvals)
SVMGeneral_error_0 = np.zeros(k)
SVMGeneral_error_1 = np.zeros(k)
SVMGeneral_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvals):
    print('When k =', k_ind)
    X_train, X_test = Xvals[train_index], Xvals[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    SVMGeneral_error_0[k_ind] = error_0
    SVMGeneral_error_1[k_ind] = error_1
    SVMGeneral_MSE[k_ind] = error_all_class
    print(classification_report(y_test, y_pred, digits=3))  
    print('The error rate of mpg_high = 0 is', error_0)
    print('The error rate of mpg_high = 1 is', error_1)
    print('The MSE is', error_all_class) 
    print()
    k_ind += 1

When k = 0
             precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

The error rate of mpg_high = 0 is 0.0
The error rate of mpg_high = 1 is 0.552083333333
The MSE is 0.540816326531

When k = 1
             precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

The error rate of mpg_high = 0 is 0.520408163265
The error rate of mpg_high = 1 is nan
The MSE is 0.520408163265

When k = 2
             precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

The error rate of mpg_high = 0 is 0.53125
The error rate of mpg_high = 1 is 0.0
The MSE is 0.5

/Users/ivychen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in long_scalars
/Users/ivychen/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# d). 

According to the result presented above, the random forest model performs the best. The MSE for the logistic regression model, random forest model and the SVM model are 0.1071, 0.0714 and 0.5077 accordingly. While SVM behaves poorly in this scenario since the prediction goes wrong more than half of the time. Then if we take a look at the prediction error for each category of mpg_high, the random forest model will provide lower error rates than the logistic model for both y=0 and y =1 cases.   